In [24]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt
import random
import pandas as pd

## Parametry klasy Pacjent:
- **id**: identyfikator pacjenta
- **arrival_time**: czas przybycia pacjenta do kliniki
- **service_start_time**: czas rozpoczęcia wizyty
- **service_end_time**: czas zakonczenia wizyty
- **room**: gabinet przypisany pacjentowi

In [25]:
class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None
        self.room = None

## Parametry klasy Gabinet:
- **id**: identyfikator gabinetu
- **env**: środowiko symulacji
- **patients_served**: ilość obsłużonych pacjentów
- **queue**: kolejka do gabinetu
- **resource**: wskazuje na obiekt z biblioteki sympy który odpowiada za faktyczne obsługiwanie symulacji

In [26]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.patients_served = 0
        self.no_show = 0
        self.queue = []
        self.resource = simpy.Resource(env, capacity=1)

## Parametry klasy Clinic:
- **curr_patient_id**: aktualny identyfikator pacjenta u żywany przy tworzeniu pacjentów
- **env**: środowsko symuacji
- **service_time**: czas obsługi jednego pacjanta
- **no_show**: prawdopodobieństwo że pacjent nie zjawi na wizytę
- **seed**: seed generatora liczby pseudolosowych
- **list_rooms**: lista gabinetów
- **processed_patients**: lista, która przechowuje informacje o pacjkentach którzy już zostali obsłużeni
- **sim_time**: całkowity czas symulacji

In [27]:
class Clinic:
    def __init__(self, number_of_rooms, service_time, no_show=0.2, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = simpy.Environment()
        self.service_time = service_time
        self.no_show = no_show
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.processed_patients = []
        self.sim_time = sim_time

        if self.seed:
            random.seed(self.seed)

Metoda **czas** przerabia czas symulacji w postaci simpy do postaci w godzinach i minutach zaczynając od godziny 8:00.

In [28]:
    def czas(self):
        hours = 8 + self.env.now//60
        minutes = self.env.now%60
        if len(str(minutes))==1:
            return f"{hours}:0{minutes}"
        return f"{hours}:{minutes}"
    Clinic.czas = czas

Metoda **generate_patients** służy do generowania pacjantów co stały czas **service_time** i losuje czy osoba się pojawi czy no show. Następnie przekazuje pacjenta do metody **serve_patient**.

In [29]:
    def generate_patients(self, room):
        def time_between_new_patients():
            return self.service_time

        while True:
            if self.env.now > self.sim_time - self.service_time:  # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break
            patient = Pacjent(id=f"{room.id}.{self.curr_patient_id}")
            patient.room = room.id
            if random.random() < self.no_show: #pacjent nie  przyszedl
                print(f'Czas {self.czas()}: Pacjent {patient.id} nie pojawił w gabinecie {room.id} w ciągu {self.service_time} minut')
                room.no_show += 1

            else: #pacjent przyszedl
                patient.arrival_time = self.env.now
                print(f"Czas {self.czas()}: Pacjent {patient.id} przybył do kliniki")
                self.env.process(self.serve_patient(patient, room))

            self.curr_patient_id += 1
            yield self.env.timeout(time_between_new_patients())
    Clinic.generate_patients = generate_patients

Metoda **serve_patient** służy do operowania kolejką do gabinetu samą obsługą pacjenta oraz jego wyjściem z gabientu.

In [30]:
    def serve_patient(self, patient, room):
        room.queue.append(patient.id)
        with room.resource.request() as request:
            yield request
            patient.service_start_time = self.env.now
            room.queue.remove(patient.id)
            print(f"Czas {self.czas()}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
            yield self.env.timeout(self.service_time)
            patient.service_end_time = self.env.now
            room.patients_served += 1
            print(f"Czas {self.czas()}: Pacjent {patient.id} wychodzi z gabinetu {room.id}")
            self.processed_patients.append(patient)
    Clinic.serve_patient = serve_patient

Metoda **run** odpowiada za uruchomienie symulacji po wywołaniu dla każdego gabinetu rozpoczyna osobny generator pacjentów.

Czas trwania jest wydłużony o 0.01 aby komunikaty które pojawiały się w ostatniej "minucie" symulacji mogły się wyświetlić.

In [31]:
    def run(self, day=1):
        for room in self.list_rooms:
            self.env.process(self.generate_patients(room))
        self.env.run(until=self.sim_time + 0.01)
        df = pd.DataFrame([
        {
            "day": day,
            "id": p.id,
            "room": p.room,
            "arrival_time": p.arrival_time,
            "service_start_time": p.service_start_time,
            "service_end_time": p.service_end_time,
            "waiting_time": p.service_start_time - p.arrival_time
        }
        for p in self.processed_patients
        ])
        df = df.set_index("id")
        return df
    Clinic.run = run

Metoda **run_multiple_times**

Przyjmuje n jako zmienną, która określa ilość wykonanych symulacji.

In [32]:
    def run_multiple_times(self, n):
        for i in range(1, n+1):
            print(f"-----------{i}-----------")
            self.env = simpy.Environment()
            self.curr_patient_id = 1
            self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(len(self.list_rooms))]
            self.processed_patients = []
            single_df = self.run(i)
            multiple_df = pd.concat([multiple_df, single_df], ignore_index=True) if i > 1 else single_df
        return multiple_df
    Clinic.run_multiple_times = run_multiple_times

## Uruchomienie symulacji:

In [49]:
clinic1 = Clinic(number_of_rooms=2, service_time=15, sim_time=120, no_show=0.4)
clinic1.run()

Czas 8:00: Pacjent 1.1 przybył do kliniki
Czas 8:00: Pacjent 2.2 nie pojawił w gabinecie 2 w ciągu 15 minut
Czas 8:00: Pacjent 1.1 wchodzi do gabinetu 1 
Czas 8:15: Pacjent 1.3 przybył do kliniki
Czas 8:15: Pacjent 2.4 przybył do kliniki
Czas 8:15: Pacjent 1.1 wychodzi z gabinetu 1
Czas 8:15: Pacjent 2.4 wchodzi do gabinetu 2 
Czas 8:15: Pacjent 1.3 wchodzi do gabinetu 1 
Czas 8:30: Pacjent 1.5 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 8:30: Pacjent 2.6 przybył do kliniki
Czas 8:30: Pacjent 2.4 wychodzi z gabinetu 2
Czas 8:30: Pacjent 1.3 wychodzi z gabinetu 1
Czas 8:30: Pacjent 2.6 wchodzi do gabinetu 2 
Czas 8:45: Pacjent 1.7 przybył do kliniki
Czas 8:45: Pacjent 2.8 przybył do kliniki
Czas 8:45: Pacjent 2.6 wychodzi z gabinetu 2
Czas 8:45: Pacjent 1.7 wchodzi do gabinetu 1 
Czas 8:45: Pacjent 2.8 wchodzi do gabinetu 2 
Czas 9:00: Pacjent 1.9 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 9:00: Pacjent 2.10 nie pojawił w gabinecie 2 w ciągu 15 minut
Czas 9:00: Pacjent 1.7 wych

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
id,,,,,,
1.1,1,1,0,0,15,0
2.4,1,2,15,15,30,0
1.3,1,1,15,15,30,0
2.6,1,2,30,30,45,0
1.7,1,1,45,45,60,0
2.8,1,2,45,45,60,0
1.11,1,1,75,75,90,0
1.13,1,1,90,90,105,0
1.15,1,1,105,105,120,0


In [50]:
sum_no_show = 0
sum_patients_served = 0
for room in clinic1.list_rooms:
    print(f"Gabinet {room.id}:")
    print(f"    Pacjentów obsłużonych: {room.patients_served}")
    print(f"    Pacjentów nieobecnych (no-show): {room.no_show}")
    print(f"    Łącznie pacjentów wygenerowanych: {room.patients_served + room.no_show}")
    sum_no_show += room.no_show
    sum_patients_served += room.patients_served
print()
print(f"Łącznie pacjentów obsłużonych: {sum_patients_served}")
print(f"Łącznie pacjentów nieobecnych (no-show): {sum_no_show}")

Gabinet 1:
    Pacjentów obsłużonych: 6
    Pacjentów nieobecnych (no-show): 2
    Łącznie pacjentów wygenerowanych: 8
Gabinet 2:
    Pacjentów obsłużonych: 3
    Pacjentów nieobecnych (no-show): 5
    Łącznie pacjentów wygenerowanych: 8

Łącznie pacjentów obsłużonych: 9
Łącznie pacjentów nieobecnych (no-show): 7


## Symulacja "na kartce"
Opiz zasad:
- szansa na no-show to 0.4
- no-show jest losowane za pomocą kostki 10d
- czas trwania symulacji to 2 h
- czas jednej wizyty to 15 min
- jest tylko 2 gabinet

**Dla gabinetu 1:**
| Czas | Zdarzenia                                                                     | Rzut kostką |
|------|-------------------------------------------------------------------------------|--------------|
| 8:00 | Pacjent 1 nie pojawił się                                                     | 2 |
| 8:15 | Pacjent 2 pojawił się i wchodzi do gabinetu                                   | 10 |
| 8:30 | Pacjent 2 wychodzi z gabinetu <br/>Pacjent 3 pojawił się i wchodzi do gabinetu | 5 |
| 8:45 | Pacjent 3 wychodzi z gabinetu <br/>Pacjent 4 pojawił się i wchodzi do gabinetu | 8 |
| 9:00 | Pacjent 4 wychodzi z gabinetu <br/>Pacjent 5 nie pojawił się                  | 4 |
| 9:15 | Pacjent 6 pojawił się i wchodzi do gabinetu                                   | 8 |
| 9:30 | Pacjent 6 wychodzi z gabinetu <br/>Pacjent 7 pojawił się i wchodzi do gabinetu | 9 |
| 9:45 | Pacjent 7 wychodzi z gabinetu <br/>Pacjent 8 nie pojawił się                  | 2 |
| 10:00 | Koniec symulacji                                                              | — |


**Dla gabinetu 2:**
| Czas | Zdarzenia                                                                     | Rzut kostką |
|------|-------------------------------------------------------------------------------|--------------|
| 8:00 | Pacjent 1 pojawił się i wchodzi do gabinetu                                   | 5 |
| 8:15 | Pacjent 1 wychodzi z gabinetu <br/> Pacjent 2 pojawił się i wchodzi do gabinetu      | 8 |
| 8:30 | Pacjent 2 wychodzi z gabinetu <br/>Pacjent 3 pojawił się i wchodzi do gabinetu | 9 |
| 8:45 | Pacjent 3 wychodzi z gabinetu <br/>Pacjent 4 pojawił się i wchodzi do gabinetu | 7 |
| 9:00 | Pacjent 4 wychodzi z gabinetu <br/>Pacjent 5 nie pojawił się                  | 1 |
| 9:15 | Pacjent 6 pojawił się i wchodzi do gabinetu                                   | 6 |
| 9:30 | Pacjent 6 wychodzi z gabinetu <br/>Pacjent 7 pojawił się i wchodzi do gabinetu | 10 |
| 9:45 | Pacjent 7 wychodzi z gabinetu <br/>Pacjent 8 pojawił się i wchodzi do gabinetu                 | 7 |
| 10:00 | Pacjent 8 wychodzi z gabinetu <br/>Koniec symulacji                                 | — |

Obsłużono 12 pacjentów, 4 pacjentów nie zjawiło się na wizycie

In [11]:
clinic.run_multiple_times(n=10)

-----------1-----------
Czas 8:00: Pacjent 1.1 przybył do kliniki
Czas 8:00: Pacjent 2.2 przybył do kliniki
Czas 8:00: Pacjent 1.1 wchodzi do gabinetu 1 
Czas 8:00: Pacjent 2.2 wchodzi do gabinetu 2 
Czas 8:15: Pacjent 1.3 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 8:15: Pacjent 2.4 przybył do kliniki
Czas 8:15: Pacjent 1.1 wychodzi z gabinetu 1
Czas 8:15: Pacjent 2.2 wychodzi z gabinetu 2
Czas 8:15: Pacjent 2.4 wchodzi do gabinetu 2 
Czas 8:30: Pacjent 1.5 przybył do kliniki
Czas 8:30: Pacjent 2.6 przybył do kliniki
Czas 8:30: Pacjent 2.4 wychodzi z gabinetu 2
Czas 8:30: Pacjent 1.5 wchodzi do gabinetu 1 
Czas 8:30: Pacjent 2.6 wchodzi do gabinetu 2 
Czas 8:45: Pacjent 1.7 nie pojawił w gabinecie 1 w ciągu 15 minut
Czas 8:45: Pacjent 2.8 przybył do kliniki
Czas 8:45: Pacjent 1.5 wychodzi z gabinetu 1
Czas 8:45: Pacjent 2.6 wychodzi z gabinetu 2
Czas 8:45: Pacjent 2.8 wchodzi do gabinetu 2 
Czas 9:00: Pacjent 1.9 przybył do kliniki
Czas 9:00: Pacjent 2.10 przybył do kliniki
Czas 9

,day,room,arrival_time,service_start_time,service_end_time,waiting_time
0,1,1,0,0,15,0
1,1,2,0,0,15,0
2,1,2,15,15,30,0
3,1,1,30,30,45,0
4,1,2,30,30,45,0
...,...,...,...,...,...,...
504,10,1,435,435,450,0
505,10,2,435,435,450,0
506,10,1,450,450,465,0
507,10,2,450,450,465,0
